In [37]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news ")
display(df)

StatementMeta(, bd589b43-b2c4-4445-8b21-28fd5a48eeef, 39, Finished, Available)

SynapseWidget(Synapse.DataFrame, 00df35eb-c6c5-4a91-8d4d-88ce8d1625b8)

In [38]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, bd589b43-b2c4-4445-8b21-28fd5a48eeef, 40, Finished, Available)

In [39]:
model=(AnalyzeText()
      .setTextCol("description")
      .setKind("SentimentAnalysis")
      .setOutputCol("response")
      .setErrorCol("error"))

StatementMeta(, bd589b43-b2c4-4445-8b21-28fd5a48eeef, 41, Finished, Available)

In [40]:
result=model.transform(df)

StatementMeta(, bd589b43-b2c4-4445-8b21-28fd5a48eeef, 42, Finished, Available)

In [41]:
display(result)

StatementMeta(, bd589b43-b2c4-4445-8b21-28fd5a48eeef, 43, Finished, Available)

SynapseWidget(Synapse.DataFrame, 6ecb74ce-5034-437a-a515-da4bc4e65b95)

In [42]:
from pyspark.sql.functions import col
sentiment_df=result.withColumn("sentiment", col("response.documents.sentiment"))


StatementMeta(, bd589b43-b2c4-4445-8b21-28fd5a48eeef, 44, Finished, Available)

In [43]:
display(sentiment_df)

StatementMeta(, bd589b43-b2c4-4445-8b21-28fd5a48eeef, 45, Finished, Available)

SynapseWidget(Synapse.DataFrame, 4a82718f-fd5f-462b-bb00-b61438d1ce68)

In [44]:
sentiment_df_final=sentiment_df.drop("error","response")

StatementMeta(, bd589b43-b2c4-4445-8b21-28fd5a48eeef, 46, Finished, Available)

In [45]:
display(sentiment_df_final)

StatementMeta(, bd589b43-b2c4-4445-8b21-28fd5a48eeef, 47, Finished, Available)

SynapseWidget(Synapse.DataFrame, b43c6838-6e78-4a99-9105-3039681462eb)

In [46]:
from pyspark.sql.functions import col,to_date
sentiment_df_final=sentiment_df_final.withColumn("datePublished",to_date(col("datePublished"),"dd-MMM-yyyy"))

StatementMeta(, bd589b43-b2c4-4445-8b21-28fd5a48eeef, 48, Finished, Available)

In [47]:
from pyspark.sql.utils import AnalysisException

table_name = 'bing_lake_db.tbl_sentiment_analysis'

# Function to check if the table exists
def table_exists(table_name):
    try:
        spark.sql(f"DESCRIBE TABLE {table_name}")
        return True
    except AnalysisException:
        return False

# Check if the table exists
if not table_exists(table_name):
    # If the table doesn't exist, create it
    sentiment_df_final.write.format("delta").saveAsTable(table_name)
else:
    # If the table exists, perform the merge operation
    print("Table Already Exists")
    sentiment_df_final.createOrReplaceTempView("vw_sentiment_df_final")

    spark.sql(f"""
        MERGE INTO {table_name} AS target_table
        USING vw_sentiment_df_final AS source_view
        ON source_view.url = target_table.url
        WHEN MATCHED AND (
            source_view.title <> target_table.title OR
            source_view.description <> target_table.description OR 
            source_view.category <> target_table.category OR 
            source_view.image <> target_table.image OR 
            source_view.provider <> target_table.provider OR 
            source_view.datePublished <> target_table.datePublished
        )
        THEN UPDATE SET
            target_table.title = source_view.title,
            target_table.description = source_view.description,
            target_table.category = source_view.category,
            target_table.image = source_view.image,
            target_table.provider = source_view.provider,
            target_table.datePublished = source_view.datePublished
        WHEN NOT MATCHED THEN
        INSERT (title, description, category, url, image, provider, datePublished)
        VALUES (source_view.title, source_view.description, source_view.category, source_view.url, source_view.image, source_view.provider, source_view.datePublished)
    """)


StatementMeta(, bd589b43-b2c4-4445-8b21-28fd5a48eeef, 49, Finished, Available)

Table Already Exists


In [48]:
display(sentiment_df_final)

StatementMeta(, bd589b43-b2c4-4445-8b21-28fd5a48eeef, 50, Finished, Available)

SynapseWidget(Synapse.DataFrame, ac2d89e4-86cd-4ae6-8076-58b685d8cd2c)